In [ ]:
import matplotlib.pyplot as plt
import h5py
import cv2 as cv
import numpy as np
import json
from matplotlib.pyplot import cm
import networkx as nx
import torch
import sys
sys.path.append('..')
import lib.scene_parser
from lib.scene_parser.rcnn.utils.visualize import select_top_predictions, select_top_pred_predictions, overlay_boxes, overlay_lines, overlay_class_names, overlay_pred_names, generate_graph, overlay_pred_boxes

In [ ]:
dataset_dict = '/mnt/f/graph-rcnn/dataset/vg_bm/'
results_dict = '/mnt/f/graph-rcnn/results/14000_2/'

roi_h5 = h5py.File(dataset_dict + 'VG-SGG.h5', "r") 
im_h5 = h5py.File(dataset_dict + 'imdb_1024.h5', 'r')

id = 12

In [ ]:
with open(dataset_dict + 'VG-SGG-dicts.json') as json_file:
    vgsgg = json.load(json_file)
    idx_to_label_str = vgsgg['idx_to_label']
idx_to_label = dict()
for key, value in idx_to_label_str.items():
    try:
        idx_to_label[int(key)] = value
    except ValueError:
        idx_to_label[float(key)] = value

In [ ]:
with open(dataset_dict + 'VG-SGG-dicts.json') as json_file:
    vgsgg = json.load(json_file)
    idx_to_predicate_str = vgsgg['idx_to_predicate']
idx_to_predicate = dict()
for key, value in idx_to_predicate_str.items():
    try:
        idx_to_predicate[int(key)] = value
    except ValueError:
        idx_to_predicate[float(key)] = value

In [ ]:
im_h5.keys()

In [ ]:
roi_h5.keys()

In [ ]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

In [ ]:
image_ids = im_h5['image_ids']
print(image_ids[62774+id])
img = im_h5['images'][62774+id]
img = img.squeeze().transpose(1,2,0)
img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
img_peview = img.copy()
img_peview_2 = img.copy()

data_split = roi_h5['split'][:]
split_mask = data_split == 2

print(np.sum(split_mask))

image_index = np.where(split_mask)[0]

split_mask = np.zeros_like(data_split).astype(bool)
split_mask[image_index] = True

all_labels = roi_h5['labels'][:, 0]
all_predicates = roi_h5['predicates'][:, 0]
all_relationships = roi_h5['relationships']
all_boxes = roi_h5['boxes_{}'.format(1024)][:]  # will index later

# convert from xc, yc, w, h to x1, y1, x2, y2
all_boxes[:, :2] = all_boxes[:, :2] - all_boxes[:, 2:] / 2
all_boxes[:, 2:] = all_boxes[:, :2] + all_boxes[:, 2:]

im_to_first_box = roi_h5['img_to_first_box'][split_mask]
im_to_last_box = roi_h5['img_to_last_box'][split_mask]
im_to_first_rel = roi_h5['img_to_first_rel'][split_mask]
im_to_last_rel = roi_h5['img_to_last_rel'][split_mask]


In [ ]:
boxes_i = all_boxes[im_to_first_box[id]:im_to_last_box[id] + 1, :]
gt_classes_i = all_labels[im_to_first_box[id]:im_to_last_box[id] + 1]
gt_predicate_i = all_predicates[im_to_first_rel[id]:im_to_last_rel[id] + 1]
relationships_i = []
for rel in range(im_to_first_rel[id],im_to_last_rel[id] + 1):
    relationships_i.append(roi_h5['relationships'][rel])
boxes_rel_i = []
for rel in relationships_i:
    boxes_rel_i.append([all_boxes[rel[0]], all_boxes[rel[1]]])

In [ ]:
roi_h5.close()
im_h5.close()

In [ ]:

colors = iter(cm.rainbow(np.linspace(0, 1, len(boxes_i))))
font = cv.FONT_HERSHEY_SIMPLEX

for num, bbox in enumerate(boxes_i):
    start_point = (round(bbox[0]), round(bbox[1]))
    end_point = (round(bbox[2]), round(bbox[3]))


    color = tuple([255*x for x in next(colors)])

    cv.rectangle(img_peview, start_point, end_point, color, 2)
    start_point = (start_point[0],start_point[1]+20)
    cv.putText(img_peview, "{}, score: {}".format(idx_to_label[gt_classes_i[num]], 'None'), start_point, font, 0.7, color, 1, cv.LINE_AA)

In [ ]:
plt.figure(figsize=(15,25))
plt.imshow(img_peview)
plt.show()

In [ ]:
# colors = iter(cm.rainbow(np.linspace(0, 1, len(boxes_i))))
# font = cv.FONT_HERSHEY_SIMPLEX

# for num, bbox in enumerate(boxes_rel_i):
#     start_point = (round(bbox[0]), round(bbox[1]))
#     end_point = (round(bbox[2]), round(bbox[3]))


#     color = tuple([255*x for x in next(colors)])

#     cv.rectangle(img_peview_2, start_point, end_point, color, 2)
#     start_point = (start_point[0],start_point[1]+20)
#     cv.putText(img_peview_2, "{}, score: {}".format(idx_to_predicate[gt_predicate_i[num]], 'None'), start_point, font, 0.7, color, 1, cv.LINE_AA)

In [ ]:
# plt.figure(figsize=(15,25))
# plt.imshow(img_peview_2)
# plt.show()

In [ ]:
def hash_tensor(tesnor):
    hash = ""
    for i in tesnor:
        hash += str(i)
    return hash

def hash_numpy(array):
    hash = ""
    for i in array:
        hash += str(i)
    return hash

In [ ]:
G = nx.DiGraph()
labeldict = {}
edge_labeldict = {}

for num, box in enumerate(boxes_i):
    box = hash_numpy(box)
    G.add_node(box)
    labeldict[box] = idx_to_label[gt_classes_i[num]]



for num, rel_box in enumerate(boxes_rel_i):
    b1 = rel_box[0]
    b2 = rel_box[1]
    n1 = None
    n2 = None
    relationships = relationships_i[num]
    rel_box = all_boxes[relationships]

    for box in boxes_i:
        box = box
        eq_1 = np.equal(b1, box)
        eq_2 = np.equal(b2, box)
        eq_all_1 = np.all(eq_1)
        eq_all_2 = np.all(eq_2)
        if eq_all_1:
            n1 = box
            pred = idx_to_predicate[gt_predicate_i[num]]
        if eq_all_2:
            n2 = box

    if n1 is not None and n2 is not None:
        h1 = hash_tensor(n1)
        h2 = hash_tensor(n2)
        label = "{}: {}".format(pred,None)
        edge_labeldict[(h1,h2)] = label
        G.add_edge(h1, h2,length=100)

pos = nx.planar_layout(G)
plt.figure(figsize=(5,10))
nx.draw(G,pos, labels=labeldict, with_labels = True)
nx.draw_networkx_edge_labels(
    G,pos,
    edge_labels=edge_labeldict,
    font_color='red'
)
plt.show()


In [ ]:
with open(results_dict + 'bbox.json') as json_file:
    bbox = json.load(json_file)
print(bbox[0])
#idx_to_label = vgsgg['idx_to_label']

In [ ]:
predictions = torch.load(results_dict + 'predictions.pth')
prediction = predictions[id]
prediction

In [ ]:
prediction.bbox

In [ ]:
prediction.extra_fields

In [ ]:
predictions_pred = torch.load(results_dict + 'predictions_pred.pth')
prediction_pred = predictions_pred[id]
prediction_pred

In [ ]:
prediction_pred.bbox

In [ ]:
prediction_pred.extra_fields

In [ ]:
top_prediction = select_top_predictions(prediction, confidence_threshold=0.0)
top_prediction.bbox

In [ ]:
"""
Select only predictions which have a `score` > self.confidence_threshold,
and returns the predictions in descending order of score
Arguments:
    predictions (BoxList): the result of the computation by the model.
        It should contain the field `scores`.
Returns:
    prediction (BoxList): the detected objects. Additional information
        of the detection properties can be found in the fields of
        the BoxList via `prediction.fields()`
"""
predictions = prediction_pred
confidence_threshold=0.4
scores = predictions.get_field("scores")
scores = scores.max(1)[0]
keep = torch.nonzero(scores > confidence_threshold).squeeze(1)
predictions = predictions[keep]
scores = scores[keep]
#scores = predictions.get_field("scores")
_, idx = scores.sort(0, descending=True)
predictions[idx]

In [ ]:
top_pred_prediction = select_top_pred_predictions(prediction_pred, confidence_threshold=0.0)
top_pred_prediction.extra_fields

In [ ]:
result = overlay_boxes(img.copy(), top_prediction)
result = overlay_class_names(result, top_prediction, idx_to_label)

plt.figure(figsize=(15,25))
plt.imshow(result)
plt.show()

In [ ]:
# result_pred = overlay_pred_boxes(img.copy(), top_pred_prediction)

# result_pred = overlay_pred_names(result_pred, top_pred_prediction, idx_to_predicate)

# plt.figure(figsize=(15,25))
# plt.imshow(result_pred)
# plt.show()

In [ ]:
predictions, pred_predictions, categories, predicates = top_prediction, top_pred_prediction, idx_to_label, idx_to_predicate

G = nx.DiGraph()
labeldict = {}
edge_labeldict = {}

scores = predictions.get_field("scores").tolist()
pred_labels = predictions.get_field("labels").tolist()
labels = [categories[i] for i in pred_labels]
pred_labels = [predicates[i] for i in pred_labels]
boxes = predictions.bbox

for num, (box, score, label) in enumerate(zip(boxes, scores, labels)):
    box = hash_tensor(box.int())
    G.add_node(box)
    labeldict[box] = label


pred_scores = pred_predictions.get_field("scores").tolist()
pred_scores = max(pred_scores)
idx_pairs = pred_predictions.get_field("idx_pairs").tolist()
predicates_pairs = [[predicates[j] for j in i] for i in idx_pairs]
pred_boxes = pred_predictions.bbox



for num, (pred_box, pred_score, predicate, box, score, label, idx_pair) in enumerate(zip(pred_boxes, pred_scores, predicates_pairs, boxes, scores, labels, idx_pairs)):
    pred_box = pred_box.int()
    b1 = pred_box[:4]
    b2 = pred_box[4:8]

    print(pred_labels[idx_pair[0]])
    print(pred_labels[idx_pair[1]])

    n1 = None
    n2 = None

    for box in boxes:
        box = box.int()
        eq_1 = torch.eq(b1, box)
        eq_2 = torch.eq(b2, box)
        eq_all_1 = torch.all(eq_1)
        eq_all_2 = torch.all(eq_2)
        if eq_all_1:
            n1 = box
            pred = predicate[0]
            score = pred_score
        if eq_all_2:
            n2 = box

    if n1 is not None and n2 is not None:
        h1 = hash_tensor(n1)
        h2 = hash_tensor(n2)
        edge_labeldict[(h1,h2)] = "{}: {}".format(pred,round(score,2))
        G.add_edge(h1, h2)

pos = nx.planar_layout(G)
plt.figure(figsize=(15,25))
nx.draw(G,pos, labels=labeldict, with_labels = True)
nx.draw_networkx_edge_labels(
    G,pos,
    edge_labels=edge_labeldict,
    font_color='red'
)
print()


In [ ]:
for num, box in enumerate(boxes):
    box = box.int()
    for pred_num, pred_box in enumerate(pred_boxes):
        pred_box_1 = pred_box.int()[:4]
        pred_box_2 = pred_box.int()[4:8]
        a = torch.eq(box, pred_box_1)
        b = torch.all(a)
        if b:
            for sub_num, bx in enumerate(boxes):
                bx = bx.int()
                a = torch.eq(pred_box_2, bx)
                b = torch.all(a)
                if b:
                    print("{}:{}:1".format(num, sub_num))
            break
        a = torch.eq(box, pred_box_2)
        b = torch.all(a)
        if b:
            for sub_num, bx in enumerate(boxes):
                bx = bx.int()
                a = torch.eq(pred_box_2, bx)
                b = torch.all(a)
                if b:
                    print("{}:{}:2".format(num, sub_num))
            break

In [ ]:
graph, labeldict, edge_labeldict = generate_graph(top_prediction, top_pred_prediction, idx_to_label, idx_to_predicate)
nx.draw(graph, labels=labeldict, with_labels = True)
# plt.savefig(os.path.join(visualize_folder, "detection_graph_{}.jpg".format(img_ids[i])))
# plt.close()

In [ ]:
G = nx.Graph()
G.add_node('A')
G.add_node('B')
G.add_edge('A','B')
nx.draw(G, with_labels=True)


In [ ]:
idx_pairs = top_pred_prediction.get_field("idx_pairs")
temp = torch.ones((idx_pairs.size(0), 1),  dtype=torch.int64)
print(temp.size())
print(idx_pairs.size())
idx_pairs = torch.cat((temp, idx_pairs), 1)
print(idx_pairs.size())
idx_pairs